In [22]:
import pandas as pd
import numpy as np

## Extract

#### <b> Load raw data </b>

In [23]:
def load() -> pd.DataFrame:
    return pd.read_excel("raw_data.xlsx")

df_raw = load()

In [24]:
display(df_raw.head())
df_raw.shape

,Date\nAnnounced,Date\nEffective,Date\nWithdrawn,Target Name,Target Industry Sector,Acquiror Name,Acquiror Industry Sector,Status,%\nsought,Target\n Net\nSales\n LTM\n($mil),...,Target\nNet Cash Fr.\n Investing\n One\n Year\n Prior,Target\nNet Cash Fr.\n Investing\n Two\n Years\n Prior,Target\nNet Cash Fr.\n Investing\n Three\n Years\n Prior,Target\nNet Cash fr.\n Financing\n One\n Year\n Prior,Target\nNet Cash fr.\n Financing\n Two\n Years\n Prior,Target\nNet Cash fr.\n Financing\n Three\n Years\n Prior,S&P 500\nClosing Price\n1 Day Before\nAnnouncement,Target\nStraight\n Debt\n One\n Year\n Prior\n($ mil),Target\nStraight\n Debt\n Two\n Years\n Prior\n($ mil),Target\nStraight\n Debt\n Three\n Years\n Prior\n($ mil)
0,2012-01-04,2012-04-02,NaT,Ascent Solar Technologies Inc,Electronic and Electrical Equipment,TFG Radiant Invest Grp Ltd,"Investment & Commodity Firms,Dealers,Exchanges",Completed,21.0,3.950,...,-3.0,6.9,-30.3,7.0,20.2,33.4,1277.06,6.6,7.3,7.1
1,2012-01-07,2012-02-13,NaT,Inhibitex Inc,Drugs,Bristol-Myers Squibb Co,Drugs,Completed,100.0,2.125,...,15.1,-5.2,14.5,0.6,21.2,-0.8,1277.81,0.3,0.7,0.8
2,2012-01-13,2012-01-13,NaT,CVR Energy Inc,Oil and Gas; Petroleum Refining,Carl Icahn,"Investment & Commodity Firms,Dealers,Exchanges",Completed,14.5,5029.113,...,-674.4,-31.3,-48.3,584.1,-31.0,-9.0,1295.50,853.9,469.0,474.7
3,2012-01-13,NaT,2012-05-04,Georgia Gulf Corp,Chemicals and Allied Products,Westlake Chemical Corp,Chemicals and Allied Products,Withdrawn,95.2,3222.884,...,-136.5,-44.6,-26.0,-85.7,-55.7,-29.1,1295.50,607.4,667.8,710.8
4,2012-01-17,2012-05-07,NaT,Convio Inc,Prepackaged Software,Blackbaud Inc,Prepackaged Software,Completed,100.0,80.353,...,-14.9,-41.7,-1.7,2.0,34.4,-2.2,1289.09,NaN,NaN,NaN


(3531, 131)

#### <b> Clean/Filter </b>

In [25]:
df_raw.columns = df_raw.columns.str.replace(r'\s+', " ", regex=True).str.strip()
df_raw = df_raw.applymap(lambda x: np.nan if x in ["np", "nm", "nan"] else x)

In [26]:
def apply_filters(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy(deep=True)
    # filter for only completed/failed mergers
    df = df[df["Status"].isin(["Completed", "Withdrawn"])]
    
    # remove share buybacks (target and acquiror are same company)
    df = df[df["Target Name"] != df["Acquiror Name"]]
    df = df[df["Date Announced"] != df["Date Effective"]]
    df = df[df["Date Announced"] != df["Date Withdrawn"]]

    # remove rows with no target price
    df = df[~df["Price Per Share"].isnull()]

    # drop rows with < 75% data filled
    drop_thresh = int(df.shape[1] * 0.75)
    df = df.dropna(thresh=drop_thresh, axis=0)
    return df

df_clean = apply_filters(df_raw)

In [27]:
df_clean.shape, df_raw.shape

((1367, 131), (3531, 131))

## Transform

#### <b> Select and parse useful columns </b>

In [28]:
cols_df = pd.read_csv("columns_to_select.csv")

print("Column type names: ", cols_df["kind"].unique())
display(cols_df)

Column type names:  ['data' 'boolean' 'ratio' 'scalar']


,column_names,kind
0,Status,data
1,Date Announced,data
2,Date Effective,data
3,Date Withdrawn,data
4,Deal Number,data
...,...,...
95,Target Net Cash fr. Financing Two Years Prior,scalar
96,Target Net Cash fr. Financing Three Years Prior,scalar
97,Target Straight Debt One Year Prior ($ mil),scalar
98,Target Straight Debt Two Years Prior ($ mil),scalar


In [29]:
df_filtered = df_clean[cols_df["column_names"].values]
df_filtered.shape

(1367, 100)

In [30]:
transformed_df = df_filtered.copy()
transformed_df.loc[:,"Status"] = np.where(transformed_df["Status"] == "Completed", 1, 0)
transformed_df = transformed_df.rename(columns={"Status":"Success"})
transformed_df.shape

(1367, 100)

#### <b> Transform time series factors </b>
- Normalise time series features into pct change features

In [31]:
def get_3y_mean_pct_change(col, col_name_list, df) -> pd.Series:
    filtered_df = df[col_name_list]
    filtered_df = filtered_df.dropna()

    filtered_df["delta_1"] = df[col_name_list[0]] / df[col_name_list[1]] - 1
    filtered_df["delta_2"] = df[col_name_list[1]] / df[col_name_list[2]] - 1

    filtered_df[col] = filtered_df[["delta_1", "delta_2"]].mean(axis=1)

    return filtered_df

col_name_grid = [
    ('Target Book Value/sh', ['Target Book Value/sh One Year Prior','Target Book Value/sh Two Years Prior','Target Book Value/sh Three Years Prior']),
    ('Target Capital Expend.',['Target Capital Expend. One Year Prior ($ mil)', 'Target Capital Expend. Two Years Prior ($ mil)','Target Capital Expend. Three Years Prior ($ mil)']),
    ( 'Target Cash & Mktbl', ['Target Cash & Mktbl. Securities One Year Prior ($ mil)', 'Target Cash & Mktbl. Securities Two Years Prior ($ mil)','Target Cash & Mktbl. Securities Three Years Prior ($ mil)']),
    ('Host Curr. Target Long Term Debt', ['Host Curr. Target Long Term Debt One Year Prior (mil)', 'Host Curr. Target Long Term Debt Two Years Prior (mil)','Host Curr. Target Long Term Debt Three Years Prior (mil)']),
    ('Target Long-Term Liab.', ['Target Long- Term Liab. 1-Year Prior ($ mil)', 'Target Long- Term Liab. 2-Years Prior ($ mil)','Target Long- Term Liab. 3-Years Prior ($ mil)']),
    ('Target Net Cash Fr. Investing', ['Target Net Cash Fr. Investing One Year Prior', 'Target Net Cash Fr. Investing Two Years Prior', 'Target Net Cash Fr. Investing Three Years Prior']),
    ('Target Net Cash fr. Financing', ['Target Net Cash fr. Financing One Year Prior','Target Net Cash fr. Financing Two Years Prior','Target Net Cash fr. Financing Three Years Prior']),
    ('Target Straight Debt', [  'Target Straight Debt One Year Prior ($ mil)','Target Straight Debt Two Years Prior ($ mil)','Target Straight Debt Three Years Prior ($ mil)'])]

# show computation results for first feature
display(get_3y_mean_pct_change(*col_name_grid[0], transformed_df))

for col, lag_cols_3y in col_name_grid:
    transformed_df[col] = get_3y_mean_pct_change(col, lag_cols_3y, transformed_df)[col]
    transformed_df = transformed_df.drop(columns=lag_cols_3y) # drop timeseries features

new_cols = [x[0] for x in col_name_grid]
display(new_cols)

,Target Book Value/sh One Year Prior,Target Book Value/sh Two Years Prior,Target Book Value/sh Three Years Prior,delta_1,delta_2,Target Book Value/sh
3,14.3,13.1,11.2,0.091603,0.169643,0.130623
4,4.2,3.3,2.8,0.272727,0.178571,0.225649
8,8.8,9.5,7.2,-0.073684,0.319444,0.122880
10,13.7,16.4,17.3,-0.164634,-0.052023,-0.108329
12,7.5,6.0,4.9,0.250000,0.224490,0.237245
...,...,...,...,...,...,...
3508,5.8,8.2,11.3,-0.292683,-0.274336,-0.283510
3515,25.2,20.3,22.4,0.241379,-0.093750,0.073815
3520,30.9,36.9,33.7,-0.162602,0.094955,-0.033823
3524,-3.6,-0.2,2.1,17.000000,-1.095238,7.952381


['Target Book Value/sh',
 'Target Capital Expend.',
 'Target Cash & Mktbl',
 'Host Curr. Target Long Term Debt',
 'Target Long-Term Liab.',
 'Target Net Cash Fr. Investing',
 'Target Net Cash fr. Financing',
 'Target Straight Debt']

In [32]:
transformed_df.shape

(1367, 84)

## Load

In [34]:
transformed_df.to_pickle("new_data.pickle")